In [1]:
import os
from dotenv import load_dotenv

import textwrap


#This is optional. I use VPN in my computer. Why I need this. 
import truststore
truststore.inject_into_ssl()



def pretty_print(*args):
    text = " ".join(str(arg) for arg in args)
    try:
        print(textwrap.fill(text, width=80))
    except Exception as e:
        print(text)  # fallback to normal print if text is not a string

        

load_dotenv('/Users/shivam13juna/Documents/scaler/iitr_classes/jan_2026/language_model_api_v2/openai_key.env')  # reads .env file in the current directory

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError(
        "OPENAI_API_KEY not found! "
        "Make sure you have a .env file with: OPENAI_API_KEY=sk-..."
    )

pretty_print("API key loaded successfully.")

API key loaded successfully.


In [2]:
from openai import OpenAI

client = OpenAI(api_key=api_key)
pretty_print("OpenAI client ready.")

OpenAI client ready.


# Responses API

## Diff b/w Chat Completions and Responses API - Structure

In [3]:

# do the same with chat completions
resp = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
		{"role": "system", "content": "You are a friendly Python tutor."},
        {"role": "user", "content": "What is a list comprehension?"}
    ]
)
pretty_print("Chat Completions output:", resp.choices[0].message.content)


Chat Completions output: A list comprehension is a concise way to create a new
list by transforming items from an existing iterable, optionally filtering them.
Basic syntax: - [expression for item in iterable] - You can add an optional
condition: [expression for item in iterable if condition]  Examples: - Squares
of numbers 0 through 9: squares = [x*x for x in range(10)] - Even numbers from 0
to 19: evens = [n for n in range(20) if n % 2 == 0] - Uppercase words: upper =
[s.upper() for s in ["apple","banana"]] - Flattening a list of lists: flat = [c
for sub in [[1,2],[3,4]] for c in sub]  Compared to a loop: - Traditional:
squares = []   for x in range(10):       squares.append(x*x) - List
comprehension does the same in one line.  Notes: - They’re great for simple
transformations and filters, but can hurt readability if too complex. - They
create a list in memory. If you want lazy evaluation, use a generator
expression: (expression for item in iterable if condition) - You can also use
d

In [4]:
resp = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a friendly Python tutor.",
    input="What is a list comprehension?"
)

pretty_print("Responses API output:", resp.output_text)


Responses API output: A list comprehension is a compact way to create a new list
by applying an expression to each item in an iterable, optionally filtering
items or nesting loops.  Common form: - Basic: [expression for item in iterable]
- With a filter: [expression for item in iterable if condition] - With multiple
loops: [expression for a in A for b in B]  Examples: - Squares: [x*x for x in
range(10)] - Evens from 0–19: [x for x in range(20) if x % 2 == 0] - Pairs: [(i,
j) for i in range(3) for j in range(2)]  Notes: - It’s equivalent to building a
list with a for-loop, but shorter and often more readable. - If you don’t want
to build a list in memory, you can use a generator expression: (expression for
item in iterable).


## Passing Multi Turn Conversation History to Responses API

In [5]:
input_messages = [
    {"role": "user", "content": "Why is Trump a jerk?"},
    {"role": "assistant", "content": "Some people are born that way."},
    {"role": "user", "content": "Name a celebrity who is not a jerk."}
]


resp = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a very candid Journalist.",
    input=input_messages
)

pretty_print("Responses API output:", resp.output_text)

Responses API output: Subjectively, many people point to these as not jerks:  -
Keanu Reeves — widely described as gracious, humble, and generous. - Tom Hanks —
known for warmth and decency in public appearances. - Dwayne “The Rock” Johnson
— often praised for positivity and philanthropy.  Of course, public personas
vary by viewer, but these are commonly viewed as pleasant, not jerky, figures.
If you want someone from a specific field (music, sports, etc.), I can tailor
suggestions.


| Feature             | Chat Completions API                             | Responses API                                              |
| ------------------- | ------------------------------------------------ | ---------------------------------------------------------- |
| **Endpoint**        | `client.chat.completions.create()`               | `client.responses.create()`                                |
| **Input format**    | `messages=[{"role": ..., "content": ...}]`       | `input=` (string or list of message dicts)                 |
| **System prompt**   | `{"role": "system", "content": ...}` in messages | `instructions=` parameter (top-level)                      |
| **Output access**   | `resp.choices[0].message.content`                | `resp.output_text`                                         |
| **Multi-turn**      | Manually pass full message history each time     | `previous_response_id=resp.id` (server-side context)       |
| **Developer role**  | Not supported (use `system`)                     | `{"role": "developer"}` for meta-instructions              |
| **Vision input**    | `{"type": "image_url", "image_url": {...}}`      | `{"type": "input_image", "image_url": ...}`                |
| **Reasoning / CoT** | Not natively supported                           | `reasoning={"effort": ..., "summary": ...}` built-in       |
| **Response object** | `ChatCompletion` with `choices[]` list           | `Response` with `output[]` list and `output_text` shortcut |
| **Streaming**       | `stream=True` yields `ChatCompletionChunk`       | `stream=True` yields server-sent events                    |
| **Tool calls**      | Supported via `tools` param                      | Supported via `tools` param (same)                         |
| **Model support**   | All chat models                                  | All chat models (newer, recommended going forward)         |


In [7]:
input_messages = [
    {"role": "user", "content": "Why is Trump a jerk?"},
    {"role": "assistant", "content": "Some people are born that way."},
    {"role": "user", "content": "Name a celebrity who is not a jerk."}
]


resp = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a very candid Journalist.",
    input=input_messages, 
    reasoning={"effort": "minimal"},  # None, low, medium, high. 
    text = {"verbosity": "low"},
    max_output_tokens=1000,
)

pretty_print("Responses API output:", resp.output_text)

Responses API output: That’s tough to prove—almost every public figure gets
flak. If you’re looking for a celebs known more for kindness or generosity,
people often point to:  - Keanu Reeves (often cited for humility and generosity)
- Dolly Parton (charitable work and warmth) - Tom Hanks (reputation for decency)
But “not a jerk” is hard to verify; public personas aren’t the full story.


link to Documentation

[Chat Completions](https://developers.openai.com/api/reference/python/resources/chat/subresources/completions/methods/create)

[Responses API](https://developers.openai.com/api/reference/python/resources/responses/methods/create)

no temperature and can not force deterministic output, but can control reasoning effort and verbosity of output.

## Key Differences: Parameters in Responses API vs Chat Completions API

### `max_tokens` → `max_output_tokens`
In the **Responses API**, the parameter to limit output length is **`max_output_tokens`**, NOT `max_tokens`.

```python
# Chat Completions API
client.chat.completions.create(model="gpt-5-nano", messages=..., max_tokens=50)

# Responses API
client.responses.create(model="gpt-5-nano", input=..., max_output_tokens=50)
```

---

### `temperature` — Not Supported on Reasoning Models (GPT-5 family)

The entire GPT-5 family (`gpt-5`, `gpt-5-mini`, `gpt-5-nano`) are **reasoning models**. They do **NOT** support `temperature` or `top_p`.

| Model Family | Type | `temperature` | `top_p` | `max_output_tokens` |
|---|---|---|---|---|
| **gpt-4o / gpt-5-nano** | Non-reasoning | ✅ Supported | ✅ Supported | ✅ Supported |
| **gpt-5 / gpt-5-mini / gpt-5-nano** | Reasoning | ❌ Not supported | ❌ Not supported | ✅ Supported |

---

### How to Influence Creativity in GPT-5 Reasoning Models

Since `temperature` is locked, you control creativity through:

**1. `reasoning.effort` parameter** — controls how deeply the model thinks:
- `"low"` → concise, more deterministic
- `"medium"` → balanced
- `"high"` → deeper reasoning, more elaborate and exploratory

```python
resp = client.responses.create(
    model="gpt-5-nano",
    reasoning={"effort": "high", "summary": "auto"},
    input="Write a creative poem about Python."
)
```

**2. Prompt Engineering** — steer creativity through instructions:
```python
resp = client.responses.create(
    model="gpt-5-nano",
    instructions="Be wildly creative. Use unexpected metaphors.",
    input="Write a poem about Python."
)
```

**Bottom line:** With GPT-5 models, creativity = `reasoning.effort` + prompt wording, not `temperature`.

# How to refer to previous response in the conversation history?

## Store = True

In [8]:
# Turn 1
message1 = 'What is a list comprehension?'


resp1 = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a friendly Python tutor.",
    input=message1,
	reasoning={"effort": "minimal"},   
    text={"verbosity": "low"}
)
pretty_print("Turn 1:", resp1.output_text)


Turn 1: A list comprehension is a compact way to create a new list by applying
an expression to each item in an iterable (like a list) and optionally filtering
items. It’s a single line alternative to a for-loop that builds a list.  Syntax:
- Basic: [expression for item in iterable] - With a filter: [expression for item
in iterable if condition]  Examples: - Squares of numbers 0–4: [x*x for x in
range(5)]   -> [0, 1, 4, 9, 16]  - Even numbers from a list: [n for n in
[1,2,3,4,5] if n % 2 == 0]   -> [2, 4]  - Transforming strings: [s.upper() for s
in ["a","b","c"]]   -> ["A", "B", "C"]  Benefits: shorter code, often faster,
readable once you’re familiar with the syntax. Use when the logic is simple; for
complex cases, a regular loop may be clearer.


In [9]:
# Turn 2 — just pass previous_response_id, no history needed!
resp2 = client.responses.create(
    model="gpt-5-nano",
    input="Can you give me an example?",
    previous_response_id=resp1.id,  # <-- this is the magic
	reasoning={"effort": "minimal"},   
    text={"verbosity": "low"}
)
pretty_print("Turn 2:", resp2.output_text) # This id is stored on open-AI server for 30 days .

Turn 2: Sure. Here’s a simple example: create a list of squared numbers from 0
to 9.  Code: squares = [x*x for x in range(10)] print(squares)  Output: [0, 1,
4, 9, 16, 25, 36, 49, 64, 81]


In [ ]:
#input_messages = [
#    {"role": "user", "content": "What is a list comprehension?"},
#    {"role": "assistant", "content": resp1.output_text},
#    {"role": "user", "content": "Can you give me an example?"}
#]

#resp2_1 = client.responses.create(
#    model="gpt-5-nano",
#    instructions="You are a friendly Python tutor.",
#    input=input_messages,
#    reasoning={"effort": "minimal"},
#    text={"verbosity": "low"}
#)
#pretty_print("Turn 2_1:", resp2_1.output_text)

In [10]:
# Turn 3 — chains from turn 2 (which already includes turn 1)
resp3 = client.responses.create(
    model="gpt-5-nano",
    input="What was my first question?",
    previous_response_id=resp2.id,
	reasoning={"effort": "minimal"},   
    text={"verbosity": "low"}
)
pretty_print("Turn 3:", resp3.output_text)

Turn 3: Your first question was: "What is a list comprehension?"


## Store = False

In [11]:
# Turn 1
resp4 = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a friendly Python tutor.",
    input="What is a list comprehension?",
	reasoning={"effort": "minimal"},   
    text={"verbosity": "low"},
	store=False
)
pretty_print("Turn 4:", resp4.output_text)

Turn 4: A list comprehension is a concise way to create a new list by applying
an expression to each item in an iterable (like a list) and optionally filtering
items. It’s a compact syntax that combines looping and building the list.  Basic
form: - [expression for item in iterable] Optional filter: - [expression for
item in iterable if condition]  Examples: - squares of numbers 0–9: [x*x for x
in range(10)] - even numbers from 0 to 19: [x for x in range(20) if x % 2 == 0]
Benefits: shorter, often faster, clearer once you’re used to the syntax. Use for
simple cases; for complex logic, a regular loop may be clearer.


In [13]:

# Turn 5 — chains from turn 4 

try:
    resp5 = client.responses.create(
        model="gpt-5-nano",
        input="What was my first question?",
        previous_response_id=resp4.id,
        reasoning={"effort": "minimal"},   
        text={"verbosity": "low"}
    )
    pretty_print("Turn 5:", resp5.output_text)
except Exception as e:
    pretty_print("Error creating response:", str(e))

Error creating response: Error code: 400 - {'error': {'message': "Previous
response with id 'resp_059f8d96db81a3bc01699dcb228188819b9f4c76fb0ed8eaaa' not
found.", 'type': 'invalid_request_error', 'param': 'previous_response_id',
'code': 'previous_response_not_found'}}


# Nature of Instruction Prompt

## Scenario 1

In [14]:
r1 = client.responses.create(
    model="gpt-5-nano",
    instructions="You are an internal support triage bot. Return only valid JSON with keys severity,suspected_causes,next_questions and do not write any customer-facing text.",
    input="A customer reports webhook deliveries started retrying heavily since 10:42 UTC and they see 502 errors from our endpoint on the Pro plan."
)

print("TURN 1:\n", r1.output_text)

TURN 1:
 {
  "severity": "critical",
  "suspected_causes": [
    "Backend webhook delivery service degradation or outage causing repeated 502 responses",
    "Recent deployment or configuration change in the webhook delivery path misconfiguring upstream integration",
    "Network connectivity issues between our delivery infrastructure and customer endpoints (DNS, TLS handshake, firewall blocks)",
    "Customer endpoints intermittently returning invalid responses during retries (leading to 502 from gateway)",
    "Gateway/edge load or misconfiguration causing a 502 when handling high retry volume"
  ],
  "next_questions": [
    "Is this affecting all subscribers or a subset of endpoints on the Pro plan?",
    "Do you have delivery IDs, timestamps, and the webhook endpoint URLs affected so we can correlate logs?",
    "Have there been any recent changes to your receiving endpoints (codes, TLS certs, firewall rules)?",
    "Are the affected endpoints in a specific region or across multipl

In [15]:
r2 = client.responses.create(
    model="gpt-5-nano",
    previous_response_id=r1.id,
    input="Now write a customer-facing email reply: apologize, explain what we’re checking, and ask for 2 specific details. Plain English, not JSON."
)

print("\nTURN 2:\n", r2.output_text)


TURN 2:
 Subject: We’re investigating the 502 errors on your webhook deliveries

Hi there,

I’m sorry for the disruption. We’re investigating why webhook deliveries on your Pro plan started retrying heavily and showing 502 errors since 10:42 UTC.

What we’re checking now:
- Our webhook delivery service health and any recent deployments or configuration changes
- Network connectivity between our delivery infrastructure and your endpoints (DNS, TLS, firewalls)
- Gateway and retry logs to understand the scope and timing of failures
- Whether your endpoints are intermittently returning errors during retries

To help us resolve this faster, could you please provide two details:
1) A few delivery IDs (or timestamps) for the 502 failures, so we can locate them in our logs
2) The exact webhook endpoint URLs that were affected (including the region, if applicable)

If you have any other details you think might help, feel free to share them.

We’ll keep you updated as we learn more. Thank you f